In [70]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [86]:
import numpy as np
import pandas as pd
import datetime as dt
from sqlalchemy import and_

# Reflect Tables into SQLAlchemy ORM

In [72]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session  
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

In [73]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [74]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [75]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [76]:
# Save references to each table
Measurement = Base.classes.measurement

Station = Base.classes.station

In [77]:
#Flask setup
app = Flask(__name__)


In [78]:
#Flask Routes
@app.route("/")
def welcome():
    """All available api routes."""
    return(
        f"Available Routes:<br/>"
        f"List of percipitation per date: /api/v1.0/precipitation<br/>"
        f"List of available stations: /api/v1.0/stations<br/>"
        f"List of temperature observations per date: /api/v1.0/tobs<br/>"
        f"List of max, min, and avg temperatures for a start date: /api/v1.0/yyyy-mm-dd<br/>"
        f"List of max, min, and avg temperatures for a start and end date: /api/v1.0/yyyy-mm-dd/yyyy-mm-dd<br/>"
    )

In [58]:
#precipitation, convert query results to a dict using date as the key and prcp as value
#use session, remember to close before for loop
@app.route("/api/v1.0/precipitation")
def precipitation():
    """Previous year of precipitation"""
    session = Session(engine)
    date_prcp = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date > '2016-08-22').\
    filter(Measurement.prcp).\
    order_by(Measurement.date).all()
    session.close()

In [79]:
@app.route("/api/v1.0/precipitation")
def precipitation_v1():
    """Previous year of precipitation"""
    session = Session(engine)
    date_prcp = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date > '2016-08-22').\
        filter(Measurement.prcp).\
        order_by(Measurement.date).all()
    session.close()

    all_prcp = {}  # Create an empty dictionary

    for date, prcp in date_prcp:
        all_prcp[date] = prcp

    return jsonify(all_prcp)

In [80]:
#stations, return jsonify list of stations from dataset
#use session, remember to close before for loop
@app.route("/api/v1.0/stations")
def stations():
    """List of stations"""
    session = Session(engine)
   
    stations = {}
    all_stations = session.query(Station.station, Station.name).all() 
    session.close()
    
    for st, name in all_stations:
       stations[st] = name
    return jsonify(stations)

In [81]:
#tobs, query dates/tobs of most active station for last year of data
#use session, remember to close before for loop
@app.route("/api/v1.0/tobs")
def tobs():
    """Previous year of temperature observations for top station"""
    session = Session(engine)
    last_12_months_station = session.query(Measurement.date, Measurement.tobs).\
    filter(Measurement.station == 'USC00519281').\
    filter(Measurement.date > '2016-08-22').\
    order_by(Measurement.station).all()
    session.close()

    all_tobs = []
    for date, tobs in last_12_months_station:
        tobs_dict = {}
        tobs_dict["date"] = date
        tobs_dict["tobs"] = tobs
        all_tobs.append(tobs_dict)
    return jsonify(all_tobs)

In [82]:
@app.route("/api/v1.0/<start>")
def start_v1(start):
    """TMIN, TAVG, TMAX per date from the starting date.
    Arguments: start (string): date format is %Y-%m-%d
    Return: TMIN, TAVE, TMAX for each date"""

    session = Session(engine)
    tobs_breakdown = session.query(
        Measurement.date,
        func.min(Measurement.tobs),
        func.avg(Measurement.tobs),
        func.max(Measurement.tobs)
    ).filter(Measurement.date >= start).group_by(Measurement.date).all()
    session.close()

    tobs_results = []

    for date, min_temp, avg_temp, max_temp in tobs_breakdown:
        results_dict = {
            "date": date,
            "min": min_temp,
            "avg": avg_temp,
            "max": max_temp
        }
        tobs_results.append(results_dict)

    return jsonify(tobs_results)


In [83]:
#given start/end date, calc TMIN, TAVG, TMAX for dates between start and end dates
#dates greater than and equal to the start date
#use session, remember to close before for loop
@app.route("/api/v1.0/<start>/<end>")
def end(start, end):
    """TMIN, TAVG, TMAX per date from the starting date to the end date.
    Arguments: 
            start (string): date format is %Y-%m-%d
            end (string): date format is %Y-%m-%d
    Return: TMIN, TAVE, TMAX for each date"""

    session = Session(engine)
    tobs_breakdown = session.query(Measurement.date,\
    func.min(Measurement.tobs),\
    func.avg(Measurement.tobs),\
    func.max(Measurement.tobs)).\
    filter(and_(Measurement.date >= '2015-01-31', Measurement.date <= '2015-02-13')).\
    group_by(Measurement.date).all()
    session.close()


In [84]:
@app.route("/api/v1.0/<start>/<end>")
def end_v1(start, end):
    """TMIN, TAVG, TMAX per date from the starting date to the end date.
    Arguments: 
            start (string): date format is %Y-%m-%d
            end (string): date format is %Y-%m-%d
    Return: TMIN, TAVE, TMAX for each date"""

    session = Session(engine)
    tobs_breakdown = session.query(
        Measurement.date,
        func.min(Measurement.tobs),
        func.avg(Measurement.tobs),
        func.max(Measurement.tobs)
    ).filter(
        and_(
            Measurement.date >= start,
            Measurement.date <= end
        )
    ).group_by(Measurement.date).all()
    session.close()

    tobs_end = []

    for date, min_temp, avg_temp, max_temp in tobs_breakdown:
        end_dict = {
            "date": date,
            "min": min_temp,
            "avg": avg_temp,
            "max": max_temp
        }
        tobs_end.append(end_dict)

    return jsonify(tobs_end)



In [87]:


# Your code for route definitions goes here

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1